# Drawing App with AI Painting Conversion

This notebook demonstrates how to create a simple drawing application using PyQt5 and convert the drawing into a famous painting using a pre-trained model from Hugging Face. The application will allow users to draw stick figures and see them transformed into paintings.


## Step 1: Import Libraries

We need to import the necessary libraries for our application.


In [7]:
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QPushButton, QLabel, QWidget
from PyQt5.QtGui import QPainter, QPen, QPixmap, QImage
from PyQt5.QtCore import Qt, QPoint
from PIL import Image
from io import BytesIO
import torch
from diffusers import StableDiffusionImg2ImgPipeline
import sys


## Step 2: Create the Drawing Canvas

We will create a `Canvas` class that allows users to draw stick figures.


In [8]:
class Canvas(QWidget):
    def __init__(self):
        super().__init__()
        self.setFixedSize(800, 500)
        self.image = QPixmap(self.size())
        self.image.fill(Qt.white)
        self.drawing = False
        self.last_point = QPoint()

    def paintEvent(self, event):
        canvas_painter = QPainter(self)
        canvas_painter.drawPixmap(self.rect(), self.image, self.image.rect())

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.drawing = True
            self.last_point = event.pos()

    def mouseMoveEvent(self, event):
        if event.buttons() & Qt.LeftButton and self.drawing:
            painter = QPainter(self.image)
            painter.setPen(QPen(Qt.black, 4, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
            painter.drawLine(self.last_point, event.pos())
            self.last_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.drawing = False

    def save_image(self, path):
        self.image.save(path)


## Step 3: Create the Main Application Window

We will create a `DrawingApp` class that includes the drawing canvas and a button to convert the drawing to a painting.


In [9]:
class DrawingApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Drawing App')
        self.setGeometry(100, 100, 800, 800)

        self.canvas = Canvas()
        main_layout = QVBoxLayout()
        main_layout.addWidget(self.canvas)

        self.painting_label = QLabel(self)
        main_layout.addWidget(self.painting_label)

        btn = QPushButton('Convert to Painting', self)
        btn.clicked.connect(self.convert_to_painting)
        main_layout.addWidget(btn)

        container = QWidget()
        container.setLayout(main_layout)
        self.setCentralWidget(container)

    def convert_to_painting(self):
        self.canvas.save_image('stick_figure.png')
        image = preprocess_image('stick_figure.png')
        painting = convert_to_painting(image)

        # Convert painting to QImage
        painting_qimage = self.pil_image_to_qimage(painting)
        self.painting_label.setPixmap(QPixmap.fromImage(painting_qimage))

    def pil_image_to_qimage(self, pil_image):
        byte_data = BytesIO()
        pil_image.save(byte_data, format="PNG")
        q_image = QImage()
        q_image.loadFromData(byte_data.getvalue())
        return q_image


## Step 4: Preprocess the Image

We need to preprocess the drawn image before sending it to the AI model.


In [10]:
def preprocess_image(image_path):
    image = Image.open(image_path)
    # Preprocess the image (resize, normalize, etc.) as required by the AI model
    return image


## Step 5: Convert the Image to a Painting

We will use a pre-trained model from Hugging Face to convert the drawing to a painting.


In [11]:
def convert_to_painting(image):
    # Load a pre-trained image-to-image translation model
    model_id = "stabilityai/stable-diffusion-2"
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
    prompt = "A beautiful painting in the style of Van Gogh"
    result = pipe(prompt=prompt, init_image=image, strength=0.75, guidance_scale=7.5)
    return result.images[0]


## Step 6: Run the Application

Finally, we will run the PyQt application within the Jupyter Notebook.


In [ ]:
def run_app():
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    window = DrawingApp()
    window.show()
    app.exec_()


# Run the application
run_app()

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]